# Question Answering over Resources

## Import Dependencies 

In [153]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI
import os
import getpass 
from bardapi import Bard
from langchain.document_loaders import PyPDFLoader
import openai

## Initialize OpenAI & Bard

In [105]:
bard_token = getpass.getpass("Enter the token for Google Bard: ")
bard = Bard(token=bard_token)

openai_key = getpass.getpass("Enter your OpenAI API Key: ")
os.environ['OPENAI_API_KEY'] = openai_key
embeddings = OpenAIEmbeddings()

## Read Files

### TXT Files

In [106]:
with open("Test-Documents/state_of_the_union.txt") as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)
db = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))])

### PDF Files

In [155]:
loader = PyPDFLoader("Test-Documents/state_of_the_union.pdf")
pages = loader.load_and_split()
db = Chroma.from_documents(pages, embeddings)

## Ask Questions

In [159]:
query = "What did president say about China?"
docs = db.similarity_search(query)

para = ""
for each in docs: 
    para += each.page_content

In [161]:
model = OpenAI(model_name="gpt-3.5-turbo-16k", temperature=0)
chain = load_qa_with_sources_chain(model, chain_type="stuff")
openai_res = chain({"input_documents": pages, "question": query}, return_only_outputs=True)

prompt = f"I want you to work as a question answering machine over given documents. The documents you will be given is the following paragraph: {para}. Then, I want to you answer the question based on the given paragraph. The question is {query}"
bard_res = bard.get_answer(prompt)['content']

/Users/yangpan/Library/Python/3.9/lib/python/site-packages/langchain/llms/openai.py:171: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/yangpan/Library/Python/3.9/lib/python/site-packages/langchain/llms/openai.py:716: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [162]:
print(openai_res['output_text'] + "\n")
print(bard_res)

The president did not mention China.
SOURCES: Test-Documents/state_of_the_union.pdf

The president said that China is a competitor and that we need to level the playing field with them. He also said that we need to pass the Bipartisan Innovation Act to make record investments in emerging technologies and American manufacturing.

Here are the specific quotes from the paragraph:

* "But to compete for the best jobs of the future, we also need to level the playing field with China and other competitors."
* "That’s why it is so important to pass the Bipartisan Innovation Act sitting in Congress that will make record investments in emerging technologies and American manufacturing."

The president did not say anything negative about China in the paragraph. He simply stated that they are a competitor and that we need to take steps to ensure that we are competitive with them.


## Language Translation Support

In [163]:
language = "Spanish"
translated = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": f"You are a {language} translator."},
        {"role": "user", "content": f"I will speak to you in any language and you will detect the language, translate it and answer in the corrected and improved version of my text, in {language}. I want you to replace my simplified A0-level words and sentences with more beautiful and elegant, upper level f{language} words and sentences. Keep the meaning same, but make them more literary. I want you to only reply the correction, the improvements and nothing else, do not write explanations. The paragrah you will translate is {bard_res}."}
    ]
)

print(translated["choices"][0]["message"]["content"])

El presidente afirmó que China es un competidor y necesitamos nivelar el campo de juego con ellos y otros competidores para poder competir por los mejores trabajos del futuro. Es por ello que es de suma importancia que se apruebe el Acta de Innovación Bipartidista en el Congreso, la cual propone realizar inversiones sin precedentes en tecnologías emergentes y la manufacturación estadounidense. El presidente no emitió comentarios negativos respecto a China en el párrafo. Simplemente, expresó la necesidad de tomar medidas para asegurar que seamos competitivos con ellos.


## Highlight Original Sources